In [ ]:
# Temporary, translations from matlab code.

In [ ]:
## New params
# init_style currently unused
options = {}
options['color_map'] = 'parula'
options['convTol'] = 15e-4

# options.xsize = [256,256];
options['maxIter'] = 2000
options['residTol'] = 5e-5
options['momentum'] = 'nesterov'
options['disp_figs'] = 1
options['disp_fig_interval'] = 40   # display image this often

options['xsize']=[z_orig, y_orig, x_orig]
options['print_interval'] = 20


options['stepsize'] = 0.7e-3

params = {}

params['tau1'] = options['stepsize']*.3e-3 # was 0.5e-7   #.000005 works pretty well for v1 camera, .0002 for v2
params['tau_soft'] = options['stepsize'] * 3e-3
tau_iso = (.25e-4)
params['z_tv_weight'] = 1    # z weighting in anisotropic TV
tau2 = .001  # Auxilliary

TVnorm3d = lambda x: sum(sum(sum(abs(x))))


In [ ]:

    
    def _A_adj_svd_3d(self, b, alpha, H_conj):
        Atb = np.sum(alpha * np.real(fft.ifft2(H_conj*fft.fft2(b))), axis=0)
        return Atb
    
    
    def reconstruct(self):
        x_init = np.zeros(z_orig, y_orig, x_orig)
        # grad_handle = @(x)linear_gradient_b(x, A, Aadj, single(b));
        grad_handle = lambda x: self._linear_gradient_b(x, A, A_adj, b.astype(np.float32))
        # prox_handle = @(x)deal(1/2*(max(x,0) + (tv3d_iso_Haar((x), params.tau1, params.z_tv_weight))), 
        #                                params.tau1*TVnorm3d(x));
        prox_handle = lambda x: (1/2 * (np.max(x, 0) + (self._tv3d_iso_Haar(x, params['tau1'], params['z_tv_weight']))), 
                                     params['tau1']*TVnorm3d(x))
        
        (x_hat, f2) = self.proxMin(grad_handle, prox_handle, x_init, b, options)

        pass
    
#     def A_svd_2d(self, H, weights, X):
#         Y = np.zeros(X.shape)
#         for r in range(0, H.shape[0]):
#             fX = fft.fft2(weights[:, :, r]*X)
#             Y = Y + (fX * H[:, :, r])
#         return crop(fft.ifft2(Y))
    
#     def A_adj_svd(self, H_conj, weights, y):
#         x = np.sum(weights * np.real(fft.ifft2(H_conj * fft.fft2(self._pad(y)))), axis=0)
#         return x
    
#     def _pad(self, M):
#         y_pad = M.shape[2] // 2
#         x_pad = M.shape[3] // 2
#         return np.pad(M, ((0, ), (0, ), (y_pad, ), (x_pad, )), mode='constant')
    
#     def _crop(self, M):
#         y_start = M.shape[2] / 2
#         y_end = M.shape[2] - y_start
#         x_start = M.shape[3] / 2
#         x_end = M.shape[3] - x_start
#         return M[:, y_start:y_end, x_start:x_end]


def _linear_gradient_b(x, A, At, b): 
    """
    Generalized gradient step for ||Ax-b|| cost function
    Takes in variable, x, linear operator, A, and adjoint, At. Does not
    require passing in Atb

    Outputs: 
    - g: Gradient. Does not need to be a vector.
    - f: Cost function value
    """
    u = A(x)
    g = At(u - b)
    f = np.linalg.norm(u[:] - b[:])




def proxMin(GradErrHandle, ProxFunc, xk, b, options):
#         if not isa(GradErrHandle,'function_handle'):
#             GradErrHandle = @(x) matrixError(GradErrHandle,transpose(GradErrHandle),x,b);
#         end
    if 'convTol' not in options:
        options['convTol'] = 1e-9
    if 'residTol' not in options:
        options['residTol'] = 1e-2
    if 'xsize' not in options:
        options['xsize'] = size(A,2)
    if 'momentum' not in options:
        options['momentum'] = 'nesterov'
    if 'disp_figs' not in options:
        options['disp_figs'] = 0

    if 'restarting' not in options:
        options['restarting'] = 0
    if 'print_interval' not in options:
        options['print_interval'] = 1
    if 'color_map' not in options:
        options['color_map'] = 'parula'
    if 'save_progress' not in options:
        options['save_progress'] = 0
    if 'restart_interval' not in options:
        options['restart_interval'] = 0
    if 'disp_crop' not in options:
        options['disp_crop'] = lambda x: x
    if 'disp_prctl' not in options:
        options['disp_prctl'] = 99.999

    # if options.save_progress
    #     if ~isfield(options,'save_progress')
    #         options.progress_file = 'prox_progress.avi';
    #     end
    #     if exist(options.progress_file,'file')
    #         overwrite_mov = input('video file exists. Overwrite? y to overwrite, n to abort.');
    #         if strcmpi(overwrite_mov,'n')
    #             new_mov_name = input('input new name (no extension): ');
    #             options.progress_file = [new_mov_name,'.avi'];
    #         end
    #     end
    #     options.vidObj = VideoWriter(options.progress_file);
    #     open(options.vidObj);
    # end
    step_num = 0
    yk = xk
    # fun_val = zeros([options.maxIter,1],'like',xk);
    fun_val = np.zeros([options['maxIter'], 1], dtype=xk.dtype)
    # step_size = options.stepsize*ones(1,'like',xk);
    step_size = options['stepsize'] * np.ones((1,), dtype=xk.dtype)
    # fm1 = zeros(1,'like',xk);
    fm1 = np.zeros((1,), dtype=xk.dtype)
    f = np.inf

    if options['momentum'] == 'linear':
        while (step_num < options['maxIter']) and (f > options['residTol']):
            step_num = step_num + 1
            (f, g) = GradErrHandle(yk)
            fun_val[step_num] = f
            x_t1 = yk - step_size*g
            yk = ProxFunc(x_t1)

            if abs(fm1-f) < options['convTol']:
                print('Answer is stable to within convTol. Stopping.\n')
                out = yk;
                break
            fm1 = f

            print('%i\t%6.4e\n'% (step_num,f))
    elif options['momentum'] == 'nesterov':
        tk = np.ones((1,), dtype=xk.dtype)
        yk = xk
        f = 1e12*ones(1,'like',xk)
        f_kp1 = f
        # tic

    while (step_num < options['maxIter']) and (f>options['residTol']):
        step_num = step_num + 1
        [f_kp1, g] = GradErrHandle(yk)
        fun_val[step_num] = f_kp1
        # fun_val(step_num) = norm(options.xin-options.crop(yk),'fro')/norm(options.xin,'fro');
        [x_kp1, norm_x] = ProxFunc(yk-options.stepsize*g)
        fun_val[step_num] = fun_val[step_num] + norm_x

        t_kp1 = (1+sqrt(1+4*tk**2))/2;
        beta_kp1 = (tk-1)/t_kp1;

        #  restart = (yk(:)-x_kp1(:))'*vec(x_kp1 - xk);  %dx(:);

        restart = (yk[:]-x_kp1[:])*vec(x_kp1 - xk)  #dx(:);
        yk = x_kp1+beta_kp1*(x_kp1 - xk);




        if step_num == 1:
            if options['known_input']:
                print('Iteration \t objective \t ||x|| \t momentum \t MSE \t PSNR\n')
            else:
                print('Iter\t ||Ax-b|| \t ||x|| \t Obj \t sparsity \t momentum \t elapsed time\n')
        if restart<0 and (step_num % options['restart_interval']) == 0:
            print('reached momentum reset interval\n')
            restart = np.inf

        if not step_num % options['print_interval']:
            if options['known_input']:
                print('%i\t %6.4e\t %6.4e\t %.3f\t %6.4e\t %.2f dB\n' % (step_num,f,norm_x,tk,norm(options.xin[:] - yk[:]), psnr(gather(yk),options.xin,255)))
            else:
#                    telapse = toc; !!!
                print('%i\t%6.4e\t%6.4e\t%6.4e\t%6.4e\t%.3f\t%.4f\n' % (step_num,f,norm_x,fun_val[step_num], nnz(x_kp1)/numel(x_kp1)*100,tk,"telapse"))

        if restart>0 and options['restarting']:
            tk = 1
            print('restarting momentum \n')
            yk = x_kp1         
        else:
            tk = t_kp1
#                %yk = y_kp1
        xk = x_kp1           
        f = f_kp1


        if abs(restart)<options['convTol']:
            print('Answer is stable to within convTol. Stopping.\n')
            out = yk
#                draw_figures(out,options)
            break
    if (f<options['residTol']):
        print('Residual below residTol. Stopping. \n')
    if step_num>=options['maxIter']:
        print('Reached max number of iterations. Stopping. \n')
    out = yk
    if nargout>1:
        varargout[1] = fun_val

    return (out, varargout)

def _tv3d_iso_Haar(x, tau, alpha):
    def ht3(x, ax, shift, thresh):
        s = np.shape(x)
        w = np.zeros(s)
        C = 1 / sqrt(2)
        if shift:
            x = np.roll(x, -1, ax)
        m = np.floor(s[ax] / 2)
